In [1]:
import torch
from torch.optim import Adam
import torch.nn as nn
from src.dataset import MIT
from src.MrCNN import MrCNN
from src.dataset import load_ground_truth
from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
from torch.utils.data import DataLoader

path = '../dataset/train_data.pth.tar'
val_path = '../dataset/val_data.pth.tar'
test_path = '../dataset/test_data.pth.tar'
dataset = MIT(dataset_path=path)
val_dataset = MIT(dataset_path=val_path)
test_dataset = MIT(dataset_path=test_path)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [3]:
test_ground_truth_path = '../dataset/test_ground_truth'
val_ground_truth_path = '../dataset/val_ground_truth'
if not Path('../dataset/test_ground_truth').exists():
    load_ground_truth(dataset=test_dataset, img_dataset_path='../dataset/ALLFIXATIONMAPS', target_folder_path=test_ground_truth_path)
if not Path('../dataset/val_ground_truth').exists():
    load_ground_truth(dataset=val_dataset, img_dataset_path='../dataset/ALLFIXATIONMAPS', target_folder_path=val_ground_truth_path)

In [21]:
# get the file in the directory of val_dataset
import matplotlib.pyplot as plt
gt = plt.imread(f'../dataset/val_ground_truth/{val_dataset.__getfile__(0)["file"]}_fixMap.jpg')
gt_height, gt_width = gt.shape[:2]
print(gt_height, gt_width)


768 1024


In [4]:
model = MrCNN().to(device)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Use Binary Cross Entropy for binary classification

In [5]:
# Training loop
for epoch in range(3):
    for batch in dataloader:
        X, y = batch
        # print(y.shape)
        # print(y.type())
        # print(y)
        X_400_crop = X[:, 0, :, :, :].to(device)
        X_250_crop = X[:, 1, :, :, :].to(device)
        X_150_crop = X[:, 2, :, :, :].to(device)
        
        optimizer.zero_grad()
        output = model(X_400_crop, X_250_crop, X_150_crop)
        #print(output)
        #print(output.shape)
        # print(output.type())
        y = y.view(-1, 1).float().to(device)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

tensor([1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 1, 0, 0])
tensor([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0